In [3]:
import cv2
import time
import os
import math
import sys
from matplotlib import pyplot as plt

sys.path.append("../modules/")

import input_output
import processor

det = processor.Processors ()

det.add_processor ("contour_extraction")
det.add_processor ("perspective_warping")

low_th  = (200, 200, 200)
high_th = (255, 255, 255)
det.add_filter (processor.inrange (low_th, high_th), "contour_extraction", "inrange rgb")
det.add_filter (processor.leave_max_area_cc (),      "contour_extraction", "max_cc_extraction")
det.add_filter (processor.contour_approximation (),  "contour_extraction", "contour_approximation")

det.add_filter (processor.warp_perspective (), "perspective_warping", "warp perspective")
det.add_filter (processor.gamma_correction (1), "perspective_warping", "gamma correction")
#det.add_filter (processor.colorspace_to_colorspace ("RGB", "HSV"),
#                "basket", "colorspaces change 1")

source  = input_output.Source ("../data/birdview.mov")
#source = input_output.Source ("../data/images/rgb_basket.jpg")
#source  = input_output.Source ("1")

out = input_output.Writer ("warped.mp4", 1400, 392)

while (True):
    _, frame = source.get_frame ()

    contour, success = det.process (frame, "contour_extraction")
    stages_contour = det.get_stages_picts ("contour_extraction")
    
    det.processors ["perspective_warping"] ["warp perspective"].set_parameters_as_contour (contour)
    warped, success = det.process (frame, "perspective_warping")
    stages_warp = det.get_stages_picts ("perspective_warping")
    
    output_frame = input_output.form_grid (stages_contour + stages_warp, window_x_sz=1400)
    cv2.imshow ("output", output_frame)
    
    #print (output_frame.shape)
    out.write (output_frame)
    
    time.sleep (0.02)
    keyb = cv2.waitKey (1) & 0xFF

    if (keyb == ord('q')):
        break

source.release()
cv2.waitKey (1)
cv2.destroyAllWindows()

video
